In [2]:
import requests
import pandas as pd

In [3]:
url_grenoble = "https://archive-api.open-meteo.com/v1/archive?latitude=45.1787&longitude=5.7148&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_toulouse = "https://archive-api.open-meteo.com/v1/archive?latitude=43.6043&longitude=1.4437&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_montpellier = "https://archive-api.open-meteo.com/v1/archive?latitude=43.6109&longitude=3.8763&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_bordeaux = "https://archive-api.open-meteo.com/v1/archive?latitude=44.8404&longitude=-0.5805&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_bayonne = "https://archive-api.open-meteo.com/v1/archive?latitude=43.4932&longitude=-1.473&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_marseille = "https://archive-api.open-meteo.com/v1/archive?latitude=43.297&longitude=5.3811&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_gap  ="https://archive-api.open-meteo.com/v1/archive?latitude=44.5586&longitude=6.0787&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"

url_lyon = "https://archive-api.open-meteo.com/v1/archive?latitude=45.7485&longitude=4.8467&start_date=1950-01-01&end_date=2025-02-01&hourly=temperature_2m&timezone=Europe%2FBerlin"



# GRENOBLE

In [ ]:
# Get response
response_gre = requests.get(url_grenoble)
response_gre = response_gre.json()
data_gre = response_gre['hourly']

# Convert to df
df_gre = pd.DataFrame(data_gre)

# Rename column
df_gre = df_gre.rename(columns={'temperature_2m':'temperature'})

# Set date format
df_gre['time'] = pd.to_datetime(df_gre['time'], utc=True)

### Temperature by day (mean)

In [23]:
# GroupBy day
df_gre_by_day = df_gre.groupby(df_gre['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_gre_by_day.to_csv("data/weather/meteo_gre_by_day.csv", index=False)
df_gre_by_day

,time,temperature
0,1950-01-01,1.645833
1,1950-01-02,1.520833
2,1950-01-03,3.595833
3,1950-01-04,5.829167
4,1950-01-05,5.741667
...,...,...
27421,2025-01-28,7.366667
27422,2025-01-29,5.512500
27423,2025-01-30,6.333333
27424,2025-01-31,5.383333


### Filter temperatures

In [77]:
# Filter temperatures < 0
df_gre['year'] = df_gre['time'].dt.year

# df_gre
df_gre_0 = df_gre[df_gre['temperature'] < 0]
# df_gre_0

occurrences_gre_0_per_year = df_gre_0.groupby('year').size().reset_index(name='nb_occurrences_0').set_index('year').fillna(0)


In [78]:
# Filter temperatures > 30
df_gre['year'] = df_gre['time'].dt.year
df_gre_30 = df_gre[df_gre['temperature'] > 30]

occurrences_gre_30_per_year = df_gre_30.groupby('year').size().reset_index(name='nb_occurrences_30').set_index('year')

In [79]:
# Merge dfs
final_df_occurrences = occurrences_gre_0_per_year.merge(occurrences_gre_30_per_year, on='year', how='outer')

final_df_occurrences['nb_occurrences_30'] = final_df_occurrences['nb_occurrences_30'].fillna(0)
final_df_occurrences['nb_occurrences_30'] = final_df_occurrences['nb_occurrences_30'].astype(int)

final_df_occurrences.to_csv("data/weather/meteo_gre_occurrences.csv")


### Temperature by year (mean)

In [125]:
df_gre_by_year = df_gre.groupby(df_gre['time'].dt.year)['temperature'].mean().reset_index()
df_gre_by_year = df_gre_by_year.iloc[:-1]
df_gre_by_year['temperature'] = round(df_gre_by_year['temperature'], 2)
df_gre_by_year

,time,temperature
0,1950,11.64
1,1951,11.36
2,1952,11.36
3,1953,11.39
4,1954,10.58
...,...,...
70,2020,14.19
71,2021,12.74
72,2022,14.83
73,2023,14.15


In [127]:
# Base 100 en 1950
base_1950 = df_gre_by_year[df_gre_by_year['time'] == 1950]['temperature'].values[0]

# Calculer l'indice base 100
df_gre_by_year['indice_base_100'] = (df_gre_by_year['temperature'] / base_1950) * 100
df_gre_by_year['indice_base_100'] = round(df_gre_by_year['indice_base_100'], 2)
df_gre_by_year.to_csv("data/weather/meteo_by_year_base100_1950.csv", index=False)

### Temperature by decade

In [137]:
# Copy df
df_gre_by_year_decade = df_gre_by_year.copy()

# Ajouter une colonne "décennie" en arrondissant l'année à la dizaine inférieure
df_gre_by_year_decade['decade'] = (df_gre_by_year_decade['time'] // 10) * 10

# GroupBy decade
df_gre_by_year_decade = df_gre_by_year_decade.groupby('decade')['temperature'].mean().reset_index()

base_decade_1950 = df_gre_by_year_decade[df_gre_by_year_decade['decade'] == 1950]['temperature'].values[0]
df_gre_by_year_decade['indice_base_100'] = round((df_gre_by_year_decade['temperature'] / base_decade_1950) * 100, 2)

df_gre_by_year_decade.to_csv("data/weather/meteo_by_year_base100_decade.csv", index=False)

### Temperature rolling average 5 years

In [151]:
df_gre_rolling = df_gre_by_year.copy()
df_gre_rolling['rolling_avg_5y'] = round(df_gre_rolling['temperature'].rolling(window=5).mean(), 2).fillna(0)

df_gre_rolling.to_csv("data/weather/meteo_by_year_rolling.csv", index=False)

df_gre_rolling

,time,temperature,indice_base_100,decade,rolling_avg_5y
0,1950,11.64,100.00,1950,0.00
1,1951,11.36,97.59,1950,0.00
2,1952,11.36,97.59,1950,0.00
3,1953,11.39,97.85,1950,0.00
4,1954,10.58,90.89,1950,11.27
...,...,...,...,...,...
70,2020,14.19,121.91,2020,13.53
71,2021,12.74,109.45,2020,13.54
72,2022,14.83,127.41,2020,13.88
73,2023,14.15,121.56,2020,13.90


### Top 5 increase - decrease

In [168]:
df_gre_max_increase_decrease = df_gre_rolling.copy()
df_gre_max_increase_decrease = df_gre_max_increase_decrease.drop(columns=['rolling_avg_5y', 'indice_base_100'])
df_gre_max_increase_decrease['temperature_change'] = df_gre_max_increase_decrease['temperature'].diff()
# df_gre_max_increase_decrease


# Top 5 increase temperature from one year to another
top5_increase = df_gre_max_increase_decrease.nlargest(5, 'temperature_change')[['time', 'temperature_change']]
top5_increase = top5_increase.rename(columns={'temperature_change':'increase', 'time': 'time_increase'})
top5_increase

,time_increase,increase
72,2022,2.09
61,2011,2.06
64,2014,1.46
14,1964,1.33
7,1957,1.32


In [169]:
# Top 5 decrease temperature from one year to another
top5_decrease = df_gre_max_increase_decrease.nsmallest(5, 'temperature_change')[['time', 'temperature_change']]
top5_decrease = top5_decrease.rename(columns={'temperature_change':'decrease', 'time': 'time_decrease'})
top5_decrease


,time_decrease,decrease
12,1962,-1.62
60,2010,-1.53
6,1956,-1.47
71,2021,-1.45
54,2004,-1.19


In [187]:
df_amplitude = df_gre.copy()
df_amplitude

,time,temperature,year
0,1950-01-01 00:00:00+00:00,-0.1,1950
1,1950-01-01 01:00:00+00:00,0.2,1950
2,1950-01-01 02:00:00+00:00,2.5,1950
3,1950-01-01 03:00:00+00:00,2.2,1950
4,1950-01-01 04:00:00+00:00,1.8,1950
...,...,...,...
658219,2025-02-01 19:00:00+00:00,3.4,2025
658220,2025-02-01 20:00:00+00:00,3.2,2025
658221,2025-02-01 21:00:00+00:00,3.2,2025
658222,2025-02-01 22:00:00+00:00,3.3,2025


# TOULOUSE

In [25]:
# Get response
response_tls = requests.get(url_toulouse)
response_tls = response_tls.json()
data_tls = response_tls['hourly']

# Convert to df
df_tls = pd.DataFrame(data_tls)

# Rename column
df_tls = df_tls.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_tls['time'] = pd.to_datetime(df_tls['time'], utc=True)
df_tls['time'] = pd.to_datetime(df_tls['time'], utc=True)
df_tls_by_day = df_tls.groupby(df_tls['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_tls_by_day.to_csv("data/weather/meteo_tls.csv", index=False)
df_tls_by_day

,time,temperature
0,1950-01-01,6.316667
1,1950-01-02,3.400000
2,1950-01-03,4.720833
3,1950-01-04,8.529167
4,1950-01-05,7.362500
...,...,...
27421,2025-01-28,7.175000
27422,2025-01-29,7.912500
27423,2025-01-30,7.504167
27424,2025-01-31,5.991667


# MONTPELLIER

In [26]:
# Get response
response_mtp = requests.get(url_montpellier)
response_mtp = response_mtp.json()
data_mtp = response_mtp['hourly']

# Convert to df
df_mtp = pd.DataFrame(data_mtp)

# Rename column
df_mtp = df_mtp.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_mtp['time'] = pd.to_datetime(df_mtp['time'], utc=True)
df_mtp['time'] = pd.to_datetime(df_mtp['time'], utc=True)
df_mtp_by_day = df_mtp.groupby(df_mtp['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_mtp_by_day.to_csv("data/weather/meteo_mtp.csv", index=False)
df_mtp_by_day

,time,temperature
0,1950-01-01,7.825000
1,1950-01-02,5.300000
2,1950-01-03,7.687500
3,1950-01-04,10.825000
4,1950-01-05,9.258333
...,...,...
27421,2025-01-28,9.604167
27422,2025-01-29,9.975000
27423,2025-01-30,10.758333
27424,2025-01-31,9.162500


# BAYONNE

In [27]:
# Get response
response_bayonne = requests.get(url_bayonne)
response_bayonne = response_bayonne.json()
data_bayonne = response_bayonne['hourly']

# Convert to df
df_bayonne = pd.DataFrame(data_bayonne)

# Rename column
df_bayonne = df_bayonne.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_bayonne['time'] = pd.to_datetime(df_bayonne['time'], utc=True)
df_bayonne['time'] = pd.to_datetime(df_bayonne['time'], utc=True)
df_bayonne_by_day = df_bayonne.groupby(df_bayonne['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_bayonne_by_day.to_csv("data/weather/meteo_bayonne.csv", index=False)
df_bayonne_by_day

,time,temperature
0,1950-01-01,7.141667
1,1950-01-02,6.029167
2,1950-01-03,8.058333
3,1950-01-04,10.837500
4,1950-01-05,8.787500
...,...,...
27421,2025-01-28,10.633333
27422,2025-01-29,10.216667
27423,2025-01-30,10.062500
27424,2025-01-31,7.783333


# MARSEILLE

In [28]:
# Get response
response_marseille = requests.get(url_marseille)
response_marseille = response_marseille.json()
data_marseille = response_marseille['hourly']

# Convert to df
df_marseille = pd.DataFrame(data_marseille)

# Rename column
df_marseille = df_marseille.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_marseille['time'] = pd.to_datetime(df_marseille['time'], utc=True)
df_marseille['time'] = pd.to_datetime(df_marseille['time'], utc=True)
df_marseille_by_day = df_marseille.groupby(df_marseille['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_marseille_by_day.to_csv("data/weather/meteo_marseille.csv", index=False)
df_marseille_by_day

,time,temperature
0,1950-01-01,8.091667
1,1950-01-02,5.691667
2,1950-01-03,8.691667
3,1950-01-04,10.975000
4,1950-01-05,9.683333
...,...,...
27421,2025-01-28,11.445833
27422,2025-01-29,10.883333
27423,2025-01-30,11.412500
27424,2025-01-31,9.425000


# GAP

In [31]:
# Get response
response_gap = requests.get(url_gap)
response_gap = response_gap.json()
data_gap = response_gap['hourly']

# Convert to df
df_gap = pd.DataFrame(data_gap)

# Rename column
df_gap = df_gap.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_gap['time'] = pd.to_datetime(df_gap['time'], utc=True)
df_gap['time'] = pd.to_datetime(df_gap['time'], utc=True)
df_gap_by_day = df_gap.groupby(df_gap['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_gap_by_day.to_csv("data/weather/meteo_gap.csv", index=False)
df_gap_by_day

,time,temperature
0,1950-01-01,-0.016667
1,1950-01-02,-4.316667
2,1950-01-03,-0.920833
3,1950-01-04,1.370833
4,1950-01-05,0.225000
...,...,...
27421,2025-01-28,4.283333
27422,2025-01-29,1.000000
27423,2025-01-30,3.525000
27424,2025-01-31,4.370833


# LYON

In [32]:
# Get response
response_lyon = requests.get(url_lyon)
response_lyon = response_lyon.json()
data_lyon = response_lyon['hourly']

# Convert to df
df_lyon = pd.DataFrame(data_lyon)

# Rename column
df_lyon = df_lyon.rename(columns={'temperature_2m':'temperature'})

# GroupBy day
df_lyon['time'] = pd.to_datetime(df_lyon['time'], utc=True)
df_lyon['time'] = pd.to_datetime(df_lyon['time'], utc=True)
df_lyon_by_day = df_lyon.groupby(df_lyon['time'].dt.date)['temperature'].mean().reset_index()

# Save to .csv
df_lyon_by_day.to_csv("data/weather/meteo_lyon.csv", index=False)
df_lyon_by_day

,time,temperature
0,1950-01-01,-0.245833
1,1950-01-02,-0.208333
2,1950-01-03,2.604167
3,1950-01-04,8.012500
4,1950-01-05,7.066667
...,...,...
27421,2025-01-28,7.366667
27422,2025-01-29,7.604167
27423,2025-01-30,7.379167
27424,2025-01-31,5.625000
